In [97]:
training_data = [
        ['Green', 3, 'Apple'], 
        ['Yellow', 3, 'Apple'], 
        ['Red', 1, 'Grape'],
        ['Red', 1, 'Grape'],
        ['Yellow', 3, 'Lemon'],
]

1. Assign headers: color, diameter, label
2. Define "unique_vals" function that find the unique values for a column in a dataset
3. 

In [98]:
header = ['color', 'size', 'label']

In [99]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

In [100]:
# Example
unique_vals(training_data, 0)

{'Green', 'Red', 'Yellow'}

In [101]:
def count_class(data):
    """Counts the number of each type of example in a dataset."""
    counts = {}
    for row in data:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [102]:
count_class(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [103]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [104]:
class Question:
    
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value
 
    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        return val == self.value
        
    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))
    

In [161]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):  # Defined above in "Question"
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [130]:
q = Question(0, "Green")
q.match(training_data[0])

True

In [131]:
true_row, false_row = partition(training_data, Question(0, 'Red'))
true_row

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [132]:
false_row

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [133]:
true_row, false_row = partition(training_data, Question(0, 'Green'))
true_row

[['Green', 3, 'Apple']]

In [134]:
false_row

[['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [111]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = count_class(rows) # {'Apple': 2, 'Grape': 2, 'Lemon': 1}
    impurity = 1
    for lbl in counts:
        prob = counts[lbl] / len(rows)
        impurity -= prob ** 2
    return impurity        

In [112]:
for i in count_class(training_data):
    print (count_class(training_data)[i])

2
2
1


In [113]:
for i in count_class(training_data):
    print (i)

Apple
Grape
Lemon


In [114]:
count_class(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [115]:
no_mixing = [['Apple'],
              ['Apple']]
gini(no_mixing)

0.0

In [116]:
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
# This will return 0.8
gini(lots_of_mixing)

0.7999999999999998

In [117]:
count_class(lots_of_mixing)

{'Apple': 1, 'Blueberry': 1, 'Grape': 1, 'Grapefruit': 1, 'Orange': 1}

In [118]:
def info_gain(left, right, current_uncertainty): 
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1-p) * gini(right)
    

In [119]:
current_uncertainty = gini(training_data)
current_uncertainty

0.6399999999999999

In [120]:
true_row,  false_row = partition(training_data, Question(0,'Green'))
info_gain(true_row, false_row, current_uncertainty)

0.1399999999999999

In [121]:
true_rows, false_rows = partition(training_data, Question(0,'Red'))
info_gain(true_rows, false_rows, current_uncertainty)

0.37333333333333324

In [143]:
# On the other hand, partitioning by Green doesn't help so much.
true_rows, false_rows = partition(training_data, Question(0,'Green'))

# We've isolated one apple in the true rows.
true_rows

[['Green', 3, 'Apple']]

In [144]:
false_rows

[['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [171]:
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1
    
    for col in range(n_features):  # 2 features with 1 label
            
            values = set([row[col] for row in rows])
            
            for val in values:
                
                question = Question(col, val)
                
                true_rows, false_rows = partition(rows, question)
                
                if len(true_rows) == 0 or len(false_rows) == 0:
                    continue
                
                gain = info_gain(true_rows, false_rows, current_uncertainty)
                
                if gain >= best_gain:
                    best_gain, best_question = gain, question
    
    return best_gain, best_question


In [172]:
best_gain, best_question = find_best_split(training_data)
#print (best_gain, best_question)
best_question

Is size >= 3?

In [173]:
true_rows0 , false_rows0  = partition(training_data, best_question)
#partition(training_data, )
best_gain1, best_question1 = find_best_split(true_rows0)
#true_rows1 , false_rows1  = partition(true_rows0, best_question1)


In [174]:
true_rows0

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [175]:
best_gain1

0.11111111111111116

In [176]:
best_question1

Is color == Yellow?

In [177]:
class Leaf:
    def __init__(self, rows):
        self.predictions = count_class(rows) # count how many in each class

In [178]:
class Decision_Node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [179]:
def build_tree(rows):
    
    gain, question = find_best_split(rows)
    
    if gain == 0:
        return Leaf(rows)
    
    true_rows , false_rows  = partition(rows, question)
    
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question, true_branch, false_branch)
    

In [180]:
def print_tree(node, spacing = ""):
    
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        
        print (spacing + "Predict", node.predictions) 
        return 
    
    print (spacing + str(node.question))
    
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [181]:
my_tree = build_tree(training_data)

In [182]:
print_tree(my_tree)

Is size >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}
